In [8]:
import os
import re
import numpy as np
import tifffile as tiff
import cv2

In [9]:
def max_projection_montage(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists
    
    # Regex to match files containing '-F4' to '-F11'
    pattern = re.compile(r'-F([4-9]|10|11)')
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif") and pattern.search(filename):
            input_path = os.path.join(input_folder, filename)
            
            # Read multi-dimensional TIF file
            img = tiff.imread(input_path)
            
            # Ensure image has correct dimensions (Z, C, H, W) or (C, Z, H, W)
            if len(img.shape) == 4:
                if img.shape[1] == 4:  # (Z, C, H, W) format
                    img = np.moveaxis(img, 1, 0)  # Convert to (C, Z, H, W)
                
                # Now img is (C, Z, H, W)
                channels = [np.max(img[i, :, :, :], axis = 0) for i in range(4)]
                
                # Normalize each channel to grayscale (0-255)
                grayscale_channels = [
                    cv2.normalize(ch, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                    for ch in channels
                ]
                
                # Arrange in a 2x2 montage
                top_row = np.hstack((grayscale_channels[0], grayscale_channels[1]))
                bottom_row = np.hstack((grayscale_channels[2], grayscale_channels[3]))
                montage = np.vstack((top_row, bottom_row))
                
                # Save montage in the output folder
                output_path = os.path.join(output_folder, filename)
                tiff.imwrite(output_path, montage)
                print(f'Processed: {filename}')
            else:
                print(f'Skipping {filename}: Unexpected image dimensions {img.shape}')
    print("Done.")

In [10]:
input_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/Test"  # Change this to your actual folder path
output_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/Test_max_split_channels"  # Change this to your actual output folder path

max_projection_montage(input_folder, output_folder)

Processed: ID0080_TK_CID_RNAiScreen_63x_ReScan_24h - R02-C02-F4_B2_IM_LUC1_init.tif
Processed: ID0080_TK_CID_RNAiScreen_63x_ReScan_24h - R02-C02-F5_B2_IM_LUC1_init.tif
Processed: ID0080_TK_CID_RNAiScreen_63x_ReScan_24h - R02-C02-F6_B2_IM_LUC1_init.tif
Processed: ID0080_TK_CID_RNAiScreen_63x_ReScan_24h - R02-C02-F7_B2_IM_LUC1_init.tif
Processed: ID0080_TK_CID_RNAiScreen_63x_ReScan_24h - R02-C02-F8_B2_IM_LUC1_init.tif
Processed: ID0080_TK_CID_RNAiScreen_63x_ReScan_24h - R02-C02-F9_B2_IM_LUC1_init.tif
Processed: ID0080_TK_CID_RNAiScreen_63x_ReScan_24h - R02-C02-F10_B2_IM_LUC1_init.tif
Processed: ID0080_TK_CID_RNAiScreen_63x_ReScan_24h - R02-C02-F11_B2_IM_LUC1_init.tif
Done.
